In [1]:
from sentence_transformers import SentenceTransformer
from transformers import GPT2Model, GPT2Tokenizer
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm.notebook import tqdm_notebook
from transformers import RobertaModel, RobertaTokenizer

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from tqdm import tqdm
import random
import numpy as np

train_data_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_sp.csv'
train_data_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_wp.csv'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [2]:
# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
embedding_model = RobertaModel.from_pretrained('roberta-base').to(device)
embedding_model.eval()  # Set the model to evaluation mode


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [3]:
from torch.utils.data import DataLoader, TensorDataset
def make_dataset(data):
    x_train = []
    y_train = []
    with torch.no_grad():
        for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
            question = row['question']
            answer = row['answer']
            distractor1 = row['distractor1']
            distractor2 = row['distractor2']

            # Tokenize and encode each pair
            q_a_input = tokenizer(question, answer, return_tensors="pt", padding=True, truncation=True).to(device)
            q_d1_input = tokenizer(question, distractor1, return_tensors="pt", padding=True, truncation=True).to(device)
            q_d2_input = tokenizer(question, distractor2, return_tensors="pt", padding=True, truncation=True).to(device)
            
            # Obtain pooled output from RoBERTa
            q_a_emb = embedding_model(**q_a_input).pooler_output
            q_d1_emb = embedding_model(**q_d1_input).pooler_output
            q_d2_emb = embedding_model(**q_d2_input).pooler_output

            # Append embeddings and labels after correct concatenation
            x_train.append(torch.cat([q_a_emb, q_a_emb], dim=1))  # Adjust as necessary
            y_train.append(torch.tensor([1], dtype=torch.float32))

            x_train.append(torch.cat([q_a_emb, q_d1_emb], dim=1))
            y_train.append(torch.tensor([0], dtype=torch.float32))

            x_train.append(torch.cat([q_a_emb, q_d2_emb], dim=1))
            y_train.append(torch.tensor([0], dtype=torch.float32))

    x_train = torch.stack(x_train).squeeze(1)
    y_train = torch.stack(y_train).view(-1, 1)
    return x_train, y_train

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(1536, 768)  # First layer to reduce dimension
        self.dropout1 = nn.Dropout(0.5)      # Dropout to prevent overfitting
        self.relu = nn.ReLU()                # ReLU activation to introduce non-linearity
        self.linear2 = nn.Linear(768, 256)   # Further reduce dimension
        self.dropout2 = nn.Dropout(0.5)      # Another dropout layer
        self.linear3 = nn.Linear(256, 1)     # Final layer to produce output
        self.sigmoid = nn.Sigmoid()          # Sigmoid activation to output a probability

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        return x



def train(model, loader, epochs):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        total_loss = 0
        for x_batch, y_batch in loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {total_loss / len(loader)}')

def inference(model, question, choices):
    model.eval()
    logits = []
    for choice in choices:
        inputs_question = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
        inputs_choice = tokenizer(choice, return_tensors="pt", padding=True, truncation=True).to(device)
        
        outputs_question = embedding_model(**inputs_question)
        outputs_choice = embedding_model(**inputs_choice)

        pooled_output_question = outputs_question.pooler_output
        pooled_output_choice = outputs_choice.pooler_output
        combined_input = torch.cat((pooled_output_question, pooled_output_choice), dim=1)

        logit = model(combined_input)
        logits.append(logit)

    probabilities = torch.cat(logits).sigmoid()
    predicted_index = torch.argmax(probabilities).item()
    return choices[predicted_index]

# Example usage
csv_data_sp = pd.read_csv(train_data_dir_sp)
csv_data_wp = pd.read_csv(train_data_dir_wp)
x_train_sp, y_train_sp = make_dataset(csv_data_sp)
x_train_wp, y_train_wp = make_dataset(csv_data_wp)

# Create a DataLoader
train_loader_sp = DataLoader(TensorDataset(x_train_sp, y_train_sp), batch_size=32, shuffle=True)
train_loader_wp = DataLoader(TensorDataset(x_train_wp, y_train_wp), batch_size=32, shuffle=True)

# Initialize and train the model
model = Model().to(device)
train(model, train_loader_sp, 10)
train(model, train_loader_wp, 10)
torch.save(model, '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/Binary-classification-version/roberta.pth')


100%|██████████| 396/396 [00:06<00:00, 63.38it/s]


Epoch 1, Loss: 0.6522629931569099
Epoch 2, Loss: 0.6520435313383738
Epoch 3, Loss: 0.6452076447506746
Epoch 4, Loss: 0.6461859482030073
Epoch 5, Loss: 0.6464430106182893
Epoch 6, Loss: 0.6474070772528648
Epoch 7, Loss: 0.6456626846144596
Epoch 8, Loss: 0.6434557028114796
Epoch 9, Loss: 0.6480086954931418
Epoch 10, Loss: 0.641064872344335
Epoch 1, Loss: 0.6394997207741988
Epoch 2, Loss: 0.6431737338241778
Epoch 3, Loss: 0.6436468347122795
Epoch 4, Loss: 0.6436065309926083
Epoch 5, Loss: 0.6357781024355638
Epoch 6, Loss: 0.6465994354925657
Epoch 7, Loss: 0.6363412533935747
Epoch 8, Loss: 0.6389836739552649
Epoch 9, Loss: 0.6436994248314908
Epoch 10, Loss: 0.637490650540904


In [4]:
import torch
from transformers import BertTokenizer, BertModel

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
embedding_model = RobertaModel.from_pretrained('roberta-base').to(device)
embedding_model.eval()  # Set the model to evaluation mode


def predict(model, df):
    for i, row in df.iterrows():
        question = row['question']
        # Ensure choices is a list; modify if necessary
        choices = eval(row['choice_list']) if isinstance(row['choice_list'], str) else row['choice_list']
        prediction = inference(model, question, choices)
        predicted_id = choices.index(prediction) if prediction in choices else -1
        df.loc[i, 'pred_id'] = int(predicted_id)
    return df


def write_answer_id(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for i, row in df.iterrows():
            f.write(f"{int(row['pred_id'])}\n")


test_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_sp.csv'
test_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_wp.csv'
test_df_sp = pd.read_csv(test_dir_sp)
test_df_wp = pd.read_csv(test_dir_wp)
test_df_sp = predict(model, test_df_sp)  # model should be your trained Bert-based model
test_df_wp = predict(model, test_df_wp) 

test_df_sp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_roberta_sp.csv")
test_df_wp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_roberta_wp.csv")




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Load your DataFrame
df_sp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_roberta_sp.csv')
df_wp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_roberta_wp.csv')

# Ensure 'id' is treated as string and replace NaN with an empty string if necessary
df_sp['id'] = df_sp['id'].astype(str)
df_wp['id'] = df_wp['id'].astype(str)

# Define function to calculate accuracy
def calculate_accuracy(df, true_label_col, pred_label_col):
    correct_predictions = df[df[true_label_col] == df[pred_label_col]]
    accuracy = len(correct_predictions) / len(df)
    return accuracy

# Calculate accuracies for each subset using the updated string column
S_ori = calculate_accuracy(df_sp[~df_sp['id'].str.contains("_")], 'label', 'pred_id')
S_sem = calculate_accuracy(df_sp[df_sp['id'].str.contains("_SR")], 'label', 'pred_id')
S_con = calculate_accuracy(df_sp[df_sp['id'].str.contains("_CR")], 'label', 'pred_id')
S_ori_sem = calculate_accuracy(df_sp[(df_sp['id'].str.contains("_SR") | ~df_sp['id'].str.contains("_"))], 'label', 'pred_id')
S_ori_sem_con = calculate_accuracy(df_sp, 'label', 'pred_id')  # For all data
S_overall = S_ori_sem_con  # Overall is the same as considering all data

W_ori = calculate_accuracy(df_wp[~df_wp['id'].str.contains("_")], 'label', 'pred_id')
W_sem = calculate_accuracy(df_wp[df_wp['id'].str.contains("_SR")], 'label', 'pred_id')
W_con = calculate_accuracy(df_wp[df_wp['id'].str.contains("_CR")], 'label', 'pred_id')
W_ori_sem = calculate_accuracy(df_wp[(df_wp['id'].str.contains("_SR") | ~df_wp['id'].str.contains("_"))], 'label', 'pred_id')
W_ori_sem_con = calculate_accuracy(df_wp, 'label', 'pred_id')  # For all data
W_overall = S_ori_sem_con  # Overall is the same as considering all data


# Print the calculated scores
print(f'S_ori: {S_ori}')
print(f'S_sem: {S_sem}')
print(f'S_con: {S_con}')
print(f'S_ori_sem: {S_ori_sem}')
print(f'S_ori_sem_con: {S_ori_sem_con}')
print(f'S_overall: {S_overall}')

# Print the calculated scores
print(f'W_ori: {W_ori}')
print(f'W_sem: {W_sem}')
print(f'W_con: {W_con}')
print(f'W_ori_sem: {W_ori_sem}')
print(f'W_ori_sem_con: {W_ori_sem_con}')
print(f'W_overall: {W_overall}')


S_ori: 0.2958579881656805
S_sem: 0.28994082840236685
S_con: 0.40236686390532544
S_ori_sem: 0.29289940828402367
S_ori_sem_con: 0.32938856015779094
S_overall: 0.32938856015779094
W_ori: 0.3787878787878788
W_sem: 0.3787878787878788
W_con: 0.4090909090909091
W_ori_sem: 0.3787878787878788
W_ori_sem_con: 0.3888888888888889
W_overall: 0.32938856015779094


SP

In [4]:
def predict(model, df):
    predictions = []
    for i, row in df.iterrows():
        question = row['question']
        choices = row['choice_list']
        predicted_choice_index = inference(model, question, choices)
        predictions.append(predicted_choice_index)  # Here you append the index, not the text
    
    return predictions

# Load your evaluation dataset into a DataFrame
# Since we can't extract the text from the image, let's assume it's already done.
eval_df = pd.DataFrame({
    'question': ["Everyone called him ""Batman,"" but he knew nothing about bats and thought they were disgusting.He still cherished being referred to as Batman! How is this possible? "],  # Example question
    'choice_list': [["He tries to be friendly.", "He is afraid others will laugh at him.", "He was the star baseball player.", "None of above."]]  # Example choices
})

# Predict the answers for the evaluation dataset
predictions = predict(model, eval_df)
print(predictions)


['None of above.']


WP

In [5]:
def predict(model, df):
    predictions = []
    for i, row in df.iterrows():
        question = row['question']
        choices = row['choice_list']
        predicted_choice_index = inference(model, question, choices)
        predictions.append(predicted_choice_index)  # Here you append the index, not the text
    
    return predictions

# Load your evaluation dataset into a DataFrame
# Since we can't extract the text from the image, let's assume it's already done.
eval_df = pd.DataFrame({
    'question': ["What kind of stock doesn't have shares?"],  # Example question
    'choice_list': [["Small-cap stock.", "Livestock.", "Growth stock.", "None of above."]]  # Example choices
})
# Predict the answers for the evaluation dataset
predictions = predict(model, eval_df)
print(predictions)

['None of above.']
